# Introduction

Score-Based Models (SBM) are models of the gradient of the log probability, $\mathbf{x}$, which we call the score
$$
    \mathbf{s}(\mathbf{x}) = \nabla_{\mathbf{x}} \log p(\mathbf{x})
$$
